<a href="https://colab.research.google.com/github/ehe363/spotify_project/blob/main/integrated_search_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
client_id = 'db905839561c4623994cf8a5f4c14f4b'
client_secret = '98ae25d2965342fe8e2c94faeb80c2b4'
redirect_uri = 'https://exampledashboard.com'
# OAuth endpoints given in the Spotify API documentation
# https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
authorization_base_url = "https://accounts.spotify.com/authorize"
token_url = "https://accounts.spotify.com/api/token"
# https://developer.spotify.com/documentation/general/guides/authorization/scopes/
scope = ["user-read-currently-playing",
         "user-top-read"]
from requests_oauthlib import OAuth2Session
spotify = OAuth2Session(client_id, scope=scope, redirect_uri=redirect_uri)
# Redirect user to Spotify for authorization
authorization_url, state = spotify.authorization_url(authorization_base_url)
print('Please go here and authorize: ', authorization_url)
# Get the authorization verifier code from the callback url
redirect_response = input('\n\nPaste the full redirect URL here: ')
from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth(client_id, client_secret)
# Fetch the access token
token = spotify.fetch_token(token_url, auth=auth, authorization_response=redirect_response)
print(token)
# Fetch a protected resource, i.e. user profile
r = spotify.get('https://api.spotify.com/v1/me')
print(r.content)

In [ ]:
top_artists = spotify.get('https://api.spotify.com/v1/me/top/artists')
top_artists_data = top_artists.content
import json
from IPython.display import Image, display
from pprint import pprint
top_artists_list = json.loads(top_artists_data)['items']
top_artists_names = []
for n in top_artists_list:
    top_artists_names.append(n["name"])

pprint(top_artists_names)

['Sheryl Crow',
 'De La Soul',
 'Green Day',
 'Bob Marley & The Wailers',
 'Peter Tosh',
 "Des'ree",
 'Bobby Shmurda',
 'Yung Gravy',
 'Tony Bennett',
 'Nas',
 'Khruangbin',
 'Daryl Hall & John Oates',
 'Kendrick Lamar',
 'Drake',
 'Rihanna',
 'Elton John',
 'Vince Staples',
 'Beyoncé',
 'SZA',
 'JAY-Z']


In [ ]:
import os

# Replace 'your_actual_api_key_here' with your real Ticketmaster API key
os.environ['TICKETMASTER_API_KEY'] = 'iPVCrQgdoj3ZTMll74BfuNig2uiYPZKS'

In [ ]:
import os

ticketmaster_api_key = os.environ.get('TICKETMASTER_API_KEY')
if ticketmaster_api_key:
    print("Ticketmaster API key is set.")
else:
    print("Ticketmaster API key is not set. Make sure to set it using os.environ.")


Ticketmaster API key is set.


In [ ]:
from IPython.core.interactiveshell import import_item
import os
import requests
from datetime import datetime, timedelta

# Retrieve API key from environment variable
API_KEY = os.environ.get('TICKETMASTER_API_KEY')
BASE_URL = 'https://app.ticketmaster.com/discovery/v2/'

def search_artist_events(artist_name):
    endpoint = f'{BASE_URL}events.json'
    params = {
        'apikey': API_KEY,
        'keyword': artist_name,
    }

    response = requests.get(endpoint, params=params)
    data = response.json()

    if response.status_code == 200:
        events = data.get('_embedded', {}).get('events', [])
        return events
    else:
        print('Error:', data.get('detail', 'Unknown error'))
        return None

def filter_events_by_date(events, days):
    today = datetime.now().date()
    end_date = today + timedelta(days=days)

    filtered_events = []
    for event in events:
        event_date = datetime.strptime(event['dates']['start']['localDate'], '%Y-%m-%d').date()
        if today <= event_date < end_date:
            filtered_events.append(event)

    return filtered_events

for item in top_artists_names:
    events = search_artist_events(item)

    if events:
        filtered_events = filter_events_by_date(events, 90)
        print(f"Found {len(filtered_events)} events for '{item}' in the next 90 days:")
        for event in filtered_events:
            event_name = event['name']
            event_date = event['dates']['start']['localDate']
            event_time = event['dates']['start'].get('localTime', 'Time not available')
            event_url = event.get('url', 'URL not available')

            print(event_name, event_date, event_time, event_url)
    else:
        print(f"No events found for '{item}'.")


Found 2 events for 'Sheryl Crow' in the next 90 days:
Sea.Hear.Now Festival 2023-09-16 Time not available https://on.fgtix.com/trk/kchM
Oceans Calling 2023-09-29 Time not available https://on.fgtix.com/trk/jpFM
Found 20 events for 'De La Soul' in the next 90 days:
The Soul Nation - Le meilleur de la Soul Pop et Motown 2023-08-26 20:30:00 https://www.ticketmaster.ca/the-soul-nation-le-meilleur-de-montreal-quebec-08-26-2023/event/31005EA3B6492995
Wu-Tang Clan & Nas: NY State Of Mind Tour 2023-09-27 20:00:00 https://www.ticketmaster.com/wutang-clan-nas-ny-state-of-brooklyn-new-york-09-27-2023/event/30005E55E61025D2
Wu-Tang Clan & Nas: NY State Of Mind Tour 2023-09-26 20:00:00 https://www.ticketmaster.com/wutang-clan-nas-ny-state-of-washington-district-of-columbia-09-26-2023/event/15005E55BBB04237
Wu-Tang Clan & Nas: NY State Of Mind Tour 2023-09-29 20:00:00 https://www.ticketmaster.com/wutang-clan-nas-ny-state-of-atlantic-city-new-jersey-09-29-2023/event/02005E58C69F6255
Wu-Tang Clan & Na